In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [2]:
train = pd.read_csv("C:/Users/derri/Documents/UNIVERSITY OF ESSEX/Year 1/Team project challenge/House prices/train.csv")
test = pd.read_csv('C:/Users/derri/Documents/UNIVERSITY OF ESSEX/Year 1/Team project challenge/House prices/test.csv')

In [3]:
train = train[train.GrLivArea < 4500]
train.reset_index(drop=True, inplace=True)

In [4]:
train_X = train.drop('SalePrice', axis=1)
train_y = train.SalePrice
test_X = test

In [6]:
onehot_train_X = pd.get_dummies(train_X)
onehot_test_X = pd.get_dummies(test_X)
train_X, test_X = onehot_train_X.align(onehot_test_X, join='left', axis=1)

In [7]:
train["SalePrice"] = np.log1p(train["SalePrice"])
y = train.SalePrice.reset_index(drop=True)
train_features = train.drop(['SalePrice'], axis=1)
test_features = test

In [8]:
#my_imputer = SimpleImputer()
#train_X = my_imputer.fit_transform(train_X)
#test_X = my_imputer.transform(test_X)

# copy of datasets
imputed_train_X = train_X.copy()
imputed_test_X = test_X.copy()

# coloums of what is being inputed
missing_val = (col for col in train_X.columns 
                                 if train_X[col].isnull().any())
for col in missing_val:
    imputed_train_X[col + 'missing'] = imputed_train_X[col].isnull()
    imputed_test_X[col + 'missing'] = imputed_test_X[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_train_X = my_imputer.fit_transform(imputed_train_X)
imputed_test_X = my_imputer.transform(imputed_test_X)

In [9]:
reg = LinearRegression()
cv_scores = cross_val_score(reg, imputed_train_X, train_y, cv=5)
print(cv_scores)

[0.8553753  0.90175022 0.87511658 0.89164747 0.86131566]


In [10]:
reg = LinearRegression()
reg.fit(imputed_train_X, train_y)
predictions = reg.predict(imputed_test_X)

In [11]:
my_submission = pd.DataFrame({'Id': test.Id, 'SalePrice':predictions})
my_submission.to_csv('BASEsubmission.csv', index=False)